In [1]:
import yfinance as yf

token = 'BTC-USD'

data = yf.download(token, interval='1d', period='max')
data = data.droplevel('Ticker', axis=1)
data.to_csv('../data/BTC-USD.csv')
data.head()

/var/folders/vp/2hwjkxh92qsdfnjr9y_xpmm00000gn/T/ipykernel_1241/1117255159.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(token, interval='1d', period='max')
[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Date,,,,,
2014-09-17,457.334015,468.174011,452.421997,465.864014,21056800
2014-09-18,424.440002,456.859985,413.104004,456.859985,34483200
2014-09-19,394.795990,427.834991,384.532013,424.102997,37919700
2014-09-20,408.903992,423.295990,389.882996,394.673004,36863600
2014-09-21,398.821014,412.425995,393.181000,408.084991,26580100
...,...,...,...,...,...
2025-08-15,117398.351562,119332.312500,116864.570312,118365.781250,68665353159
2025-08-16,117491.351562,117996.062500,117271.953125,117398.421875,48036922378
2025-08-17,117453.062500,118595.773438,117279.523438,117492.789062,45852169525


In [2]:
import pandas as pd
from ta.trend import EMAIndicator, MACD
from ta.momentum import RSIIndicator

# Carga tus datos en 'data' con columna 'Close'
data = pd.read_csv('../data/BTC-USD.csv')

# EMAs
data['ema20'] = EMAIndicator(data['Close'], window=20).ema_indicator()
data['ema50'] = EMAIndicator(data['Close'], window=50).ema_indicator()

# RSIs
data['rsi10']  = RSIIndicator(data['Close'], window=10).rsi()
data['rsi30']  = RSIIndicator(data['Close'], window=30).rsi()
data['rsi200'] = RSIIndicator(data['Close'], window=200).rsi()

# MACD
macd_ind = MACD(data['Close'], window_fast=12, window_slow=26, window_sign=9)
data['macd']   = macd_ind.macd()
data['signal'] = macd_ind.macd_signal()
data['hist']   = macd_ind.macd_diff()

# Señal de subida de precio
data['Price-Up'] = (data['Close'].shift(-1) > data['Close']).astype(int)

# Limpieza y exportación
data.dropna(inplace=True)
data.to_csv('../data/BTC-I.csv', index=False)
data
print(data.tail())


            Date          Close           High            Low           Open  \
3985  2025-08-15  117398.351562  119332.312500  116864.570312  118365.781250   
3986  2025-08-16  117491.351562  117996.062500  117271.953125  117398.421875   
3987  2025-08-17  117453.062500  118595.773438  117279.523438  117492.789062   
3988  2025-08-18  116252.312500  117614.171875  114723.679688  117453.906250   
3989  2025-08-19  113756.601562  116709.281250  113644.398438  116270.000000   

           Volume          ema20          ema50      rsi10      rsi30  \
3985  68665353159  117541.958236  114829.431484  49.285988  54.304271   
3986  48036922378  117537.138553  114933.820507  49.594012  54.409111   
3987  45852169525  117529.131310  115032.614311  49.456598  54.355996   
3988  72787808090  117407.529518  115080.445612  45.102138  52.687407   
3989  67299610624  117059.822094  115028.530159  37.480990  49.425172   

         rsi200         macd      signal        hist  Price-Up  
3985  54.729665

# Stream_Crypto_Data 

In [7]:
import json 
import websocket
import pandas as pd
import numpy as np

coin = 'btcusdt@kline_1m'
socket = 'wss://stream.binance.com:9443/stream?streams=' + coin
print(socket)


ModuleNotFoundError: No module named 'websocket'

In [ ]:
def extract_ohlcv(source):
    event_time = pd.to_datetime(source['data']['E'], unit='ms')
    open = source['data']['k']['o']
    high = source['data']['k']['h']
    low = source['data']['k']['l']
    close = source['data']['k']['c']   
    volume = source['data']['k']['v']

    data = {'Open': open, 'High': high, 'Low': low, 'Close': close, 'Volume': volume}

    df = pd.DataFrame(data, index=[event_time])
    df.index.name = 'timestamp'
    df = df.astype(float)
    df = df.reset_index()
    print(df)
    return df

def on_message(ws, message):
    message = json.loads(message)
    extract_ohlcv(message)

In [ ]:
ws = websocket.WebSocketApp(socket, on_message=on_message)
ws.run_forever()

                timestamp      Open     High       Low     Close    Volume
0 2025-02-27 17:53:18.014  84573.53  84593.7  84494.26  84494.26  10.41565
                timestamp      Open     High       Low     Close    Volume
0 2025-02-27 17:53:20.015  84573.53  84593.7  84494.26  84494.27  10.43499
                timestamp      Open     High       Low     Close    Volume
0 2025-02-27 17:53:22.029  84573.53  84593.7  84494.26  84494.27  10.61498


True